# 00. 더미 데이터 생성기

이 노트북은 파이프라인 테스트를 위한 더미 무역 데이터를 생성합니다.

**핵심 요구사항:**
- 43개월의 시계열 데이터
- 100개 품목 (`item_code`)
- 의도적인 공행성 쌍 주입 (예: `item_B(t) = f(item_A(t-k)) + noise`)

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys

# 프로젝트 루트를 경로에 추가
sys.path.append(str(Path.cwd().parent))
from config import Config

# 재현성을 위한 시드 설정
np.random.seed(Config.SEED)

## 1. 기본 설정

In [ ]:
# 데이터 파라미터
N_MONTHS = 43
N_ITEMS = 100
START_DATE = '2022-01-01'

# 날짜 범위 생성
date_range = pd.date_range(start=START_DATE, periods=N_MONTHS, freq='MS')

print(f"기간: {date_range[0]} ~ {date_range[-1]}")
print(f"총 {N_MONTHS}개월, {N_ITEMS}개 품목")

## 2. 기본 품목 데이터 생성

트렌드 + 계절성 + 노이즈 패턴 사용

In [ ]:
def generate_base_series(n_periods, trend_slope=0, seasonal_amplitude=0, noise_std=1):
    """
    트렌드, 계절성, 노이즈를 가진 기본 시계열 생성
    
    Parameters:
    -----------
    n_periods : int
        시계열 길이
    trend_slope : float
        선형 트렌드 기울기
    seasonal_amplitude : float
        계절성 진폭
    noise_std : float
        노이즈 표준편차
    """
    t = np.arange(n_periods)
    
    # 트렌드
    trend = trend_slope * t
    
    # 계절성 (12개월 주기)
    seasonal = seasonal_amplitude * np.sin(2 * np.pi * t / 12)
    
    # 노이즈
    noise = np.random.normal(0, noise_std, n_periods)
    
    # 기본값 설정 (최소값 0)
    base_value = 100
    
    return np.maximum(0, base_value + trend + seasonal + noise)

# 80개의 독립 품목 생성
n_independent = 80
independent_series = {}

for i in range(n_independent):
    item_code = f"item_{i:02d}"
    
    # 각 품목에 랜덤한 특성 부여
    trend_slope = np.random.uniform(-0.5, 0.5)
    seasonal_amplitude = np.random.uniform(5, 20)
    noise_std = np.random.uniform(3, 10)
    
    independent_series[item_code] = generate_base_series(
        N_MONTHS, trend_slope, seasonal_amplitude, noise_std
    )

print(f"{n_independent}개의 독립 품목 생성 완료")

## 3. 의도적 공행성 쌍 생성

**목표:** 공행성 탐지 알고리즘 검증을 위한 ground truth 생성

**패턴:**
- **CCF 검증용:** `item_B(t) = 0.7 * item_A(t-2) + noise`
- **Granger 검증용:** `item_B(t) = 0.6 * item_A(t-1) + 0.3 * item_A(t-3) + noise`
- **DTW 검증용:** 비선형 변환 + 가변 시차

In [ ]:
comovement_series = {}
comovement_pairs = []  # ground truth 기록

# 패턴 1: 단순 시차 상관 (CCF 검증용) - 10쌍
for i in range(10):
    leader_idx = i
    follower_idx = 80 + i
    
    leader_code = f"item_{leader_idx:02d}"
    follower_code = f"item_{follower_idx:02d}"
    
    lag = np.random.randint(1, 4)  # 1~3개월 시차
    coefficient = np.random.uniform(0.6, 0.9)
    
    # 선행 품목 시계열
    leader_series = independent_series[leader_code]
    
    # 후행 품목 생성 (시차 적용)
    follower_series = np.zeros(N_MONTHS)
    follower_series[:lag] = generate_base_series(lag, noise_std=5)[:lag]
    follower_series[lag:] = coefficient * leader_series[:-lag] + np.random.normal(0, 5, N_MONTHS - lag)
    follower_series = np.maximum(0, follower_series)
    
    comovement_series[follower_code] = follower_series
    comovement_pairs.append({
        'leader': leader_code,
        'follower': follower_code,
        'lag': lag,
        'coefficient': coefficient,
        'pattern': 'simple_lag'
    })

# 패턴 2: 다중 시차 (Granger 검증용) - 5쌍
for i in range(5):
    leader_idx = 10 + i
    follower_idx = 90 + i
    
    leader_code = f"item_{leader_idx:02d}"
    follower_code = f"item_{follower_idx:02d}"
    
    lag1, lag2 = 1, 3
    coef1, coef2 = 0.5, 0.3
    
    leader_series = independent_series[leader_code]
    
    follower_series = np.zeros(N_MONTHS)
    follower_series[:lag2] = generate_base_series(lag2, noise_std=5)[:lag2]
    follower_series[lag2:] = (
        coef1 * leader_series[lag2-lag1:-lag1] + 
        coef2 * leader_series[:-lag2] + 
        np.random.normal(0, 5, N_MONTHS - lag2)
    )
    follower_series = np.maximum(0, follower_series)
    
    comovement_series[follower_code] = follower_series
    comovement_pairs.append({
        'leader': leader_code,
        'follower': follower_code,
        'lags': [lag1, lag2],
        'coefficients': [coef1, coef2],
        'pattern': 'multi_lag'
    })

# 패턴 3: 비선형 변환 (DTW 검증용) - 5쌍
for i in range(5):
    leader_idx = 15 + i
    follower_idx = 95 + i
    
    leader_code = f"item_{leader_idx:02d}"
    follower_code = f"item_{follower_idx:02d}"
    
    leader_series = independent_series[leader_code]
    
    # 비선형 변환 (제곱근)
    lag = 2
    follower_series = np.zeros(N_MONTHS)
    follower_series[:lag] = generate_base_series(lag, noise_std=5)[:lag]
    follower_series[lag:] = np.sqrt(leader_series[:-lag] + 1) * 10 + np.random.normal(0, 3, N_MONTHS - lag)
    follower_series = np.maximum(0, follower_series)
    
    comovement_series[follower_code] = follower_series
    comovement_pairs.append({
        'leader': leader_code,
        'follower': follower_code,
        'lag': lag,
        'pattern': 'nonlinear'
    })

print(f"{len(comovement_pairs)}개의 공행성 쌍 생성 완료")
print(f"\nGround Truth:")
for pair in comovement_pairs[:3]:
    print(f"  {pair['leader']} -> {pair['follower']} ({pair['pattern']})")

## 4. 전체 데이터 결합 및 저장

In [ ]:
# 모든 시계열 결합
all_series = {**independent_series, **comovement_series}

# Long format DataFrame 생성
data_records = []
for item_code, values in all_series.items():
    for date, value in zip(date_range, values):
        data_records.append({
            'date': date,
            'item_code': item_code,
            'value': value
        })

df_long = pd.DataFrame(data_records)

# 저장
output_path = Config.DATA_RAW / "dummy_trade_data.csv"
df_long.to_csv(output_path, index=False)

print(f"✓ 데이터 저장 완료: {output_path}")
print(f"\n데이터 요약:")
print(df_long.head())
print(f"\nShape: {df_long.shape}")
print(f"날짜 범위: {df_long['date'].min()} ~ {df_long['date'].max()}")
print(f"품목 수: {df_long['item_code'].nunique()}")

## 5. Ground Truth 저장 (공행성 쌍)

In [ ]:
# Ground truth를 CSV로 저장
import json

ground_truth_path = Config.DATA_RAW / "comovement_ground_truth.json"
with open(ground_truth_path, 'w') as f:
    json.dump(comovement_pairs, f, indent=2)

print(f"✓ Ground truth 저장 완료: {ground_truth_path}")

## 6. 데이터 검증

In [ ]:
import matplotlib.pyplot as plt

# Wide format으로 변환하여 시각화
df_wide = df_long.pivot(index='date', columns='item_code', values='value')

# 공행성 쌍 시각화 (첫 번째 쌍)
first_pair = comovement_pairs[0]
leader = first_pair['leader']
follower = first_pair['follower']

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(df_wide.index, df_wide[leader], label=f"{leader} (Leader)", marker='o')
ax.plot(df_wide.index, df_wide[follower], label=f"{follower} (Follower, lag={first_pair['lag']})", marker='s')
ax.legend()
ax.set_title("공행성 쌍 예시")
ax.set_xlabel("날짜")
ax.set_ylabel("값")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("✓ 더미 데이터 생성 완료!")